# 📞 Call Center Performance Analysis

This notebook analyzes agent performance in a call center by calculating success rates, total calls made, and visualizing the results using Python libraries such as Pandas, Matplotlib, and Seaborn.

In [ ]:
# 📦 Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# 📁 Load the Dataset
# Load the dataset
file_path = r"C:\Users\user\Documents\PYTHON PROJECTS\Call Center\Data\Sales and Marketing Call Center.csv"
df = pd.read_csv(file_path)

In [ ]:
# Create a full name column to enhance identification
df['Agent_Full_Name'] = df['Agent_First_Name'] + ' ' + df['Agent_Last_Name']

# Convert 'Agent_Rating' to numeric for accurate filtering
df['Agent_Rating'] = pd.to_numeric(df['Agent_Rating'], errors='coerce')

# Filter and display top-performing agents with a perfect rating of 5.0
top_agents = df[df['Agent_Rating'] == 5.0][['Agent_Full_Name', 'Agent_Rating']].drop_duplicates()

# Display the result
print("Top-Performing Agents Based on Agent Rating:")
print(top_agents)

In [ ]:
# 📊 Average Call Duration by Each Agent
# Ensure 'Call_Duration_Minutes' is numeric
df['Call_Duration_Minutes'] = pd.to_numeric(df['Call_Duration_Minutes'], errors='coerce')

# Group by 'Agent_Full_Name' and calculate the average call duration, then sort in descending order
average_call_duration = df.groupby('Agent_Full_Name')['Call_Duration_Minutes'].mean().round(2).reset_index()
average_call_duration.rename(columns={'Call_Duration_Minutes': 'Average_Call_Duration'}, inplace=True)
average_call_duration = average_call_duration.sort_values(by='Average_Call_Duration', ascending=False)

# Print the heading
print(f"{'Agent Full Name':<30} | {'Average Call Duration (minutes)':<10}")
print("-" * 50)

# Display each agent's full name followed by their average call duration with proper spacing
for _, row in average_call_duration.iterrows():
  print(f"{row['Agent_Full_Name']:<30} | {row['Average_Call_Duration']:<10} minutes")

In [ ]:
# 📌 Agents with the Highest Success Rate in Closing Calls
# Ensure 'Call_Outcome' column is properly labeled and 'Success' is counted
df['Call_Outcome'] = df['Call_Outcome'].str.strip()  # To handle any extra spaces if present

# Filter out the successful calls
success_calls = df[df['Call_Outcome'] == 'Success']

# Group by 'Agent_Full_Name' and calculate the total and successful calls
agent_success = success_calls.groupby('Agent_Full_Name').size().reset_index(name='Total_Successful_Calls')

# Calculate total calls per agent
total_calls = df.groupby('Agent_Full_Name').size().reset_index(name='Total_Calls')

# Merge the dataframes to have total and successful calls together
agent_data = pd.merge(agent_success, total_calls, on='Agent_Full_Name')

# Calculate the success rate as a percentage
agent_data['Success_Rate'] = (agent_data['Total_Successful_Calls'] / agent_data['Total_Calls']) * 100

# Round the success rate to 2 decimal places
agent_data['Success_Rate'] = agent_data['Success_Rate'].round(2)

# Sort the data by Success Rate in descending order
agent_data_sorted = agent_data.sort_values(by='Success_Rate', ascending=False)

# Print the heading
print(f"{'Agent Full Name':<30} | {'Total Successful Calls':<20} | {'Success Rate (%)':<15}")
print("-" * 70)

# Display each agent's full name, total successful calls, and success rate
for _, row in agent_data_sorted.iterrows():
  print(f"{row['Agent_Full_Name']:<30} | {row['Total_Successful_Calls']:<20} | {row['Success_Rate']:<15}")

In [ ]:
# 📈 Success Rate Visualization
# Ensure 'Call_Outcome' column is cleaned
df['Call_Outcome'] = df['Call_Outcome'].str.strip()

# Calculate total successful calls
success_calls = df[df['Call_Outcome'] == 'Success']
agent_success = success_calls.groupby('Agent_Full_Name').size().reset_index(name='Total_Successful_Calls')

# Calculate total calls for each agent
total_calls = df.groupby('Agent_Full_Name').size().reset_index(name='Total_Calls')

# Merge and compute success rate
agent_data = pd.merge(agent_success, total_calls, on='Agent_Full_Name')
agent_data['Success_Rate'] = (agent_data['Total_Successful_Calls'] / agent_data['Total_Calls']) * 100
agent_data['Success_Rate'] = agent_data['Success_Rate'].round(2)

# Sort data for visualization
agent_data_sorted = agent_data.sort_values(by='Success_Rate', ascending=False)

# Create the plot
plt.figure(figsize=(14, 8))
sns.set_style("whitegrid")

# Bar plot for successful calls
bar_plot = sns.barplot(
  x='Agent_Full_Name',
  y='Total_Successful_Calls',
  data=agent_data_sorted,
  color='skyblue',
  edgecolor='black'
)

# Line plot for success rate
line_plot = bar_plot.twinx()
line_plot.plot(agent_data_sorted['Agent_Full_Name'], agent_data_sorted['Success_Rate'], 
             marker='o', color='red', label='Success Rate (%)')

# Annotations for success rates
for i, rate in enumerate(agent_data_sorted['Success_Rate']):
  line_plot.annotate(f"{rate}%", (i, rate), textcoords="offset points", xytext=(0, 5), ha='center', color='red')

# Labels and title
bar_plot.set_xlabel("Agent Full Name", fontsize=12)
bar_plot.set_ylabel("Total Successful Calls", fontsize=12)
line_plot.set_ylabel("Success Rate (%)", fontsize=12)
plt.title("Top-Performing Agents: Successful Calls and Success Rate (%)", fontsize=14)
bar_plot.tick_params(axis='x', rotation=45)

# Layout and show plot
plt.tight_layout()
plt.show()

In [ ]:

# Calculate total calls made by each agent, sorted in descending order
total_calls = df.groupby('Agent_Full_Name').size().reset_index(name='Total_Calls')
total_calls_sorted = total_calls.sort_values(by='Total_Calls', ascending=False)

# Display the result
print("Agent Full Name                Total Calls Made")
print("-------------------------------------------------")
for i, row in total_calls_sorted.iterrows():
  print(f"{row['Agent_Full_Name']:<30} {row['Total_Calls']:,.0f}")

In [ ]:
# 🔢 Are Agent Ratings Correlated with Call Outcomes?
# Step 1: Group by agent and calculate total calls, successful calls, and average ratings
agent_performance = df.groupby(['Agent_First_Name', 'Agent_Last_Name']).agg(
  total_calls=('Call_Outcome', 'size'),
  successful_calls=('Call_Outcome', lambda x: (x == 'Success').sum()),
  avg_rating=('Agent_Rating', 'mean')
).reset_index()

# Step 2: Calculate the success rate for each agent
agent_performance['success_rate'] = agent_performance['successful_calls'] / agent_performance['total_calls']

# Step 3: Calculate the correlation between agent ratings and success rate
correlation = agent_performance['avg_rating'].corr(agent_performance['success_rate'])

# Print the correlation value
print(f"Correlation between Agent Rating and Call Outcome Success Rate: {correlation:.4f}")

In [ ]:

          # Step 1: Add a new column for full names
          agent_performance['Agent_Full_Name'] = agent_performance['Agent_First_Name'] + ' ' + agent_performance['Agent_Last_Name']
          
          # Step 2: Create the scatter plot
          plt.figure(figsize=(12, 8))
          
          # Plot scatter and add a regression line
          sns.regplot(x='avg_rating', y='success_rate', data=agent_performance, logistic=False, scatter_kws={'s': 60, 'color': 'blue'}, line_kws={'color': 'red'})
          
          # Annotate each data point with the agent's full name in vertical orientation
          for i in range(len(agent_performance)):
              # Add a vertical space between points and the agent names by adjusting the y position
              y_offset = 0.02  # Adjust this value to control the amount of space
              plt.text(agent_performance['avg_rating'].iloc[i], 
                       agent_performance['success_rate'].iloc[i] + y_offset,  # Adding offset to y position
                       agent_performance['Agent_Full_Name'].iloc[i], 
                       fontsize=9, ha='center', va='bottom', rotation=90)
          
          # Add titles and labels
          plt.title('Relationship Between Agent Rating and Call Outcome Success Rate', fontsize=16)
          plt.xlabel('Average Agent Rating', fontsize=12)
          plt.ylabel('Call Outcome Success Rate', fontsize=12)
          
          # Show the plot
          plt.show()